# Simple Agent with LangChain 1.0's `create_agent`

## Overview
This notebook demonstrates **LangChain 1.0's new `create_agent` function** - the simplified way to build agents without needing to manually construct graphs.

## What is `create_agent`?
`create_agent` is LangChain 1.0's recommended way to build agents for most use cases. It provides:
- **Simplified API**: No need to manually build state graphs
- **Built on LangGraph**: Gets all LangGraph benefits (streaming, persistence, etc.)
- **Middleware support**: Advanced context engineering capabilities
- **Production-ready**: Designed for real applications

## When to Use `create_agent` vs LangGraph
- **Use `create_agent`**: Single agent, straightforward tool use, standard workflows
- **Use LangGraph**: Complex multi-step orchestration, custom routing logic, multi-agent systems

## Requirements
- **LangChain 1.0+**: `pip install langchain>=1.0`
- **API Key**: OpenAI or Ollama
- **Tavily API**: For search tool (free tier available)

## What You'll Learn
1. How to create an agent with `create_agent`
2. How to add tools to your agent
3. How to invoke and stream agent responses
4. How to add middleware for advanced control

## Step 1: Import Libraries

First, let's import the necessary components.

In [2]:
import os
from dotenv import load_dotenv

# LangChain 1.0 imports
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

# Load environment variables
load_dotenv(".env")

print("✓ Libraries imported successfully")
print("✓ Using LangChain 1.0's create_agent function")

✓ Libraries imported successfully
✓ Using LangChain 1.0's create_agent function


## Step 2: Set Up Tools

Let's define the tools our agent can use. We'll start with a search tool.

In [3]:
# Initialize search tool
search_tool = TavilySearchResults(
    max_results=3,
    description="Search the web for current information. Use this when you need up-to-date facts."
)

# Collect all tools
tools = [search_tool]

print(f"✓ {len(tools)} tool(s) configured")
print(f"  - {search_tool.name}: {search_tool.description}")

✓ 1 tool(s) configured
  - tavily_search_results_json: Search the web for current information. Use this when you need up-to-date facts.


/var/folders/k7/8m84jpf52x7gvx628v82h2p80000gn/T/ipykernel_8942/3110299831.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(


## Step 3: Create the Agent (Basic Version)

Now let's create our agent using `create_agent`. This is much simpler than manually building a graph!

In [4]:
# Create the agent - it's this simple!
agent = create_agent(
    model="gpt-4o-mini",
    tools=tools,
    system_prompt="You are a helpful research assistant. Use the search tool when you need current information."
)

print("✓ Agent created successfully!")
print("  Model: gpt-4o-mini")
print(f"  Tools: {len(tools)}")
print("  Ready to use!")

✓ Agent created successfully!
  Model: gpt-4o-mini
  Tools: 1
  Ready to use!


## Step 4: Invoke the Agent

Let's test our agent with a question that requires current information.

In [5]:
# Ask a question
query = "What are the latest AI developments in 2025?"

print(f"🔍 Query: {query}\n")
print("⚙️  Agent is working...\n")

# Invoke the agent
result = agent.invoke({
    "messages": [{"role": "user", "content": query}]
})

# Display the response
print("=" * 80)
print("AGENT RESPONSE:")
print("=" * 80)
print(result["messages"][-1].content)
print("\n" + "=" * 80)

🔍 Query: What are the latest AI developments in 2025?

⚙️  Agent is working...

AGENT RESPONSE:
Here are some of the latest AI developments and trends observed in 2025:

1. **Advanced Learning Systems**: AI systems are now capable of continuous learning, allowing them to adapt based on recent interactions and updates without needing full retraining. This includes developments in large language models (LLMs) that can reason more like humans and manage complex tasks in real-time.

   - [Source: Morgan Stanley](https://www.morganstanley.com/insights/articles/ai-trends-reasoning-frontier-models-2025-tmt)

2. **Near-Infinite Memory**: AI technologies, such as Google's Gemini, are utilizing advanced memory capabilities that allow for ongoing conversations and the recall of past interactions over long periods. This leads to highly personalized and context-aware responses, enhancing user experience significantly.

   - [Source: Forbes](https://www.forbes.com/councils/forbestechcouncil/2025/04/

## Step 5: Stream Agent Responses

For better user experience, we can stream the agent's responses in real-time.

In [6]:
query = "What are the top tech companies by market cap in 2025?"

print(f"🔍 Query: {query}\n")
print("📡 Streaming response...\n")
print("=" * 80)

# Stream the response
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="updates"
):
    for node_name, node_output in chunk.items():
        if "messages" in node_output:
            message = node_output["messages"][-1]
            if hasattr(message, "content") and message.content:
                print(f"\n[{node_name}]:")
                print(message.content[:200] + "..." if len(message.content) > 200 else message.content)

print("\n" + "=" * 80)

🔍 Query: What are the top tech companies by market cap in 2025?

📡 Streaming response...


[tools]:
[{"title": "Top tech companies by market cap 2025", "url": "https://www.statista.com/statistics/1350976/leading-tech-companies-worldwide-by-market-cap/?srsltid=AfmBOoqxAyjX-FuMez7ui-_oSFzuPmzxsOEOFHEN...

[model]:
As of October 2025, the top tech companies by market capitalization are as follows:

1. **NVIDIA** - $5,060 billion
2. **Apple** - $4,015 billion
3. **Microsoft** - $4,006 billion
4. **Alphabet (Googl...



## Step 6: Test Without Tools

Let's see how the agent behaves when it doesn't need external tools.

In [7]:
# Simple question that doesn't need search
simple_query = "What is 25 * 4?"

print(f"🔍 Query: {simple_query}\n")

result = agent.invoke({
    "messages": [{"role": "user", "content": simple_query}]
})

print("Response:", result["messages"][-1].content)
print("\n✓ Agent answered directly without using tools!")

🔍 Query: What is 25 * 4?

Response: 25 * 4 equals 100.

✓ Agent answered directly without using tools!


## Step 7: Add Custom Tools

Let's create a custom tool and add it to our agent.

In [8]:
from langchain.tools import tool

@tool
def calculate(expression: str) -> str:
    """Evaluates a mathematical expression. Input should be a valid Python math expression."""
    try:
        result = eval(expression)
        return f"The result is: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Create a new agent with both tools
enhanced_agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_tool, calculate],
    system_prompt="You are a helpful assistant with search and calculation capabilities."
)

print("✓ Enhanced agent created with 2 tools:")
print("  - Search tool")
print("  - Calculator tool")

✓ Enhanced agent created with 2 tools:
  - Search tool
  - Calculator tool


In [9]:
# Test the calculator tool
calc_query = "What is 15% of 250?"

print(f"🔍 Query: {calc_query}\n")

result = enhanced_agent.invoke({
    "messages": [{"role": "user", "content": calc_query}]
})

print("Response:", result["messages"][-1].content)

🔍 Query: What is 15% of 250?

Response: 15% of 250 is 37.5.


## Step 8: Add Middleware (Advanced)

LangChain 1.0's killer feature is **middleware** - hooks that let you control agent behavior.

In [10]:
from langchain.agents.middleware import ModelCallLimitMiddleware

# Limit the number of model calls to prevent runaway costs
call_limiter = ModelCallLimitMiddleware(
    run_limit=5,  # Max 5 LLM calls per invocation
    exit_behavior="end"  # Gracefully end if limit reached
)

# Create agent with middleware
safe_agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_tool],
    middleware=[call_limiter],
    system_prompt="You are a helpful research assistant."
)

print("✓ Agent created with middleware protection")
print("  - Max 5 model calls per run")
print("  - Automatic cost control")

✓ Agent created with middleware protection
  - Max 5 model calls per run
  - Automatic cost control


In [11]:
# Test the safe agent
result = safe_agent.invoke({
    "messages": [{"role": "user", "content": "Tell me about recent breakthroughs in quantum computing"}]
})

print("Response:", result["messages"][-1].content[:300] + "...")

Response: Recent breakthroughs in quantum computing in 2023 have been significant and transformative, spanning both theoretical advancements and practical applications. Here are some key highlights:

1. **Shift to Practical Implementations**: There has been a notable transition in the quantum computing field ...


## Step 9: Conversation with Memory

Let's maintain a conversation across multiple turns.

In [12]:
# Start a conversation
conversation_history = []

def chat(query: str):
    """Send a message and get a response"""
    conversation_history.append({"role": "user", "content": query})
    
    result = agent.invoke({"messages": conversation_history})
    
    assistant_message = result["messages"][-1]
    conversation_history.append({
        "role": "assistant",
        "content": assistant_message.content
    })
    
    return assistant_message.content

print("💬 Starting conversation...\n")

💬 Starting conversation...



In [13]:
# First message
response1 = chat("What is the capital of France?")
print("User: What is the capital of France?")
print(f"Agent: {response1}\n")

User: What is the capital of France?
Agent: The capital of France is Paris.



In [14]:
# Follow-up question (tests memory)
response2 = chat("What's the population of that city?")
print("User: What's the population of that city?")
print(f"Agent: {response2}\n")

User: What's the population of that city?
Agent: As of recent estimates, the population of the city of Paris is approximately **2,206,488**. However, when considering the larger urban agglomeration, the population rises to about **11,346,800**. 

For more details, you can check the sources:
- [World Population Review](https://worldpopulationreview.com/cities/france/paris)
- [City Population](https://www.citypopulation.de/en/france/paris/paris/75056__paris/)



In [15]:
# View conversation history
print("\n📝 Full Conversation History:")
print("=" * 80)
for i, msg in enumerate(conversation_history, 1):
    role = msg["role"].capitalize()
    content = msg["content"][:100] + "..." if len(msg["content"]) > 100 else msg["content"]
    print(f"{i}. {role}: {content}\n")


📝 Full Conversation History:
1. User: What is the capital of France?

2. Assistant: The capital of France is Paris.

3. User: What's the population of that city?

4. Assistant: As of recent estimates, the population of the city of Paris is approximately **2,206,488**. However,...



## Step 10: Using Ollama (Local LLM Alternative)

You can also use local models with `create_agent`.

In [ ]:
from langchain_ollama import ChatOllama

# Create agent with local model
local_llm = ChatOllama(model="mistral-nemo:12b", temperature=0)

local_agent = create_agent(
    model=local_llm,
    tools=[calculate],  # Just calculator for local testing
    system_prompt="You are a helpful math assistant."
)

print("✓ Local agent created with Ollama")
print("  Model: mistral-nemo:12b")
print("  Running 100% locally!")

In [ ]:
# Test local agent
result = local_agent.invoke({
    "messages": [{"role": "user", "content": "Calculate the square root of 144"}]
})

print("Response:", result["messages"][-1].content)

## Comparison: `create_agent` vs Manual LangGraph

### `create_agent` (LangChain 1.0)
✅ **Pros:**
- Simple, concise API
- Fast to prototype
- Built-in middleware support
- Automatic tool routing
- Good for 80% of use cases

⚠️ **Cons:**
- Less control over routing logic
- Black-box behavior for complex flows
- Harder to customize graph structure

### Manual LangGraph
✅ **Pros:**
- Full control over graph structure
- Custom routing logic
- Complex multi-agent orchestration
- Conditional edges and loops
- Great for complex workflows

⚠️ **Cons:**
- More verbose code
- Steeper learning curve
- More boilerplate

### When to Use Each

| Use Case | Recommended Approach |
|----------|---------------------|
| Simple Q&A with tools | `create_agent` |
| Research assistant | `create_agent` |
| Customer support bot | `create_agent` |
| Multi-step workflows | LangGraph |
| Multi-agent systems | LangGraph |
| Complex conditional routing | LangGraph |
| Custom error handling | LangGraph |

## Key Takeaways

### What We Learned
✅ **`create_agent` simplicity**: Create powerful agents in just a few lines  
✅ **Tool integration**: Easy to add search, calculations, and custom tools  
✅ **Middleware power**: Advanced control without complexity  
✅ **Streaming support**: Real-time responses for better UX  
✅ **Memory management**: Built-in conversation history  

### Best Practices
1. **Start simple**: Use `create_agent` first, switch to LangGraph if needed
2. **Clear system prompts**: Guide your agent's behavior
3. **Tool descriptions**: Help the agent understand when to use each tool
4. **Add middleware**: Use limiters to prevent runaway costs
5. **Test both modes**: Try streaming and regular invoke

### Next Steps
1. **Add more tools**: File operations, database queries, API calls
2. **Explore middleware**: Guardrails, summarization, human-in-the-loop
3. **Try checkpointers**: Persist conversations across sessions
4. **Learn LangGraph**: For complex orchestration needs
5. **Build production apps**: Add error handling, logging, monitoring

### Resources
- [LangChain 1.0 Docs](https://docs.langchain.com/)
- [create_agent API Reference](https://python.langchain.com/docs/modules/agents/)
- [Middleware Guide](https://docs.langchain.com/oss/python/langchain/middleware)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)